In [ ]:
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from transformers import pipeline
import torch
import torchaudio
from gtts import gTTS
import os
from keybert import KeyBERT
from IPython.display import Audio
from googletrans import Translator
import json

In [ ]:
#!pip install keybert

In [ ]:
#!pip install googletrans==3.1.0a0

In [ ]:
!pip install gtts

In [ ]:
def fetch_news(company, max_articles=10):
    """Fetches news articles from Bing News for a given company."""
    url = f'https://www.bing.com/news/search?q={company}'
    headers = {
       'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
       'Accept-language': 'en-US, en; q=0.9',
       'connection':'keep-alive',
       'upgrade-insecure-requests': '1'
        }

    #fetching the big news page
    doc = requests.get(url, headers=headers)

    #parsing the html content
    soup = BeautifulSoup(doc.text, 'html.parser')
    articles = []

    #finding articles and extracting title & summary
    for item in soup.find_all('div', class_='newsitem', limit=max_articles):
        title_tag = item.find('a', class_='title')
        summary_tag = item.find('div', class_='snippet')

        if title_tag and summary_tag:
            articles.append({
                'Title': title_tag.text.strip(),
                'Summary': summary_tag.text.strip(),
            })
    return articles

In [ ]:

def analyze_sentiment(articles):
    for article in articles:
        summary = article['Summary']
        blob = TextBlob(summary)
        polarity = blob.sentiment.polarity
        if polarity > 0:
            sentiment = 'positive'
        elif polarity < 0:
            sentiment = 'negative'
        else:
            sentiment = 'neutral'
        article['Sentiment'] = sentiment
    return articles

In [ ]:
# Load the Zero-Shot classification model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def classify_topics(articles):
    """Uses zero-shot classification to assign topics to news articles."""
    candidate_labels = [
        "Technology", "Electric Vehicles", "Stock Market", "Finance", "Innovation",
        "Regulation", "Economy", "Artificial Intelligence", "Climate Change", "Energy",
        "Consulting", "Business Strategy", "Consumer Behavior", "Leadership", "Cryptocurrency",
        "Healthcare", "Cybersecurity", "Retail", "Manufacturing", "Supply Chain"
    ]

    for article in articles:
        summary = article['Summary']
        result = classifier(summary, candidate_labels, multi_label=True)
        article['Topics'] = result["labels"][:3]

    return articles


Device set to use cuda:0


In [ ]:
def compute_sentiment_distribution(articles):
    """Counts the number of articles in each sentiment category."""
    sentiment_counts = {"Positive": 0, "Negative": 0, "Neutral": 0}

    for article in articles:
        sentiment = article["Sentiment"].capitalize()
        if sentiment in sentiment_counts:
            sentiment_counts[sentiment] += 1

    return sentiment_counts


In [ ]:
# Initialize KeyBERT model for keyword extraction
kw_model = KeyBERT()

def extract_keywords(text, top_n=3):
    """Extracts key phrases from the given text."""
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english')
    return ', '.join([kw[0] for kw in keywords[:top_n]])

def compare_coverage(articles):
    """Generates a comparative analysis of news articles with concise comparisons."""
    comparisons = []
    topics_set = [set(article['Topics']) for article in articles]

    for i in range(len(articles) - 1):  # Iterate sequentially across all articles
        article1, article2 = articles[i], articles[i + 1]

        # Extract concise title keywords
        title1_keywords = extract_keywords(article1['Title'])
        title2_keywords = extract_keywords(article2['Title'])

        # Identify the main difference in focus
        comparison = {
            "Comparison": f"Article {i+1} highlights {title1_keywords}, while Article {i+2} discusses {title2_keywords}.",
            "Impact": f"The first article highlights {article1['Sentiment']} sentiment, whereas the second is {article2['Sentiment']}."
        }

        # Topic Overlap Analysis
        common_topics = topics_set[i] & topics_set[i + 1]
        unique_1 = topics_set[i] - topics_set[i + 1]
        unique_2 = topics_set[i + 1] - topics_set[i]

        comparison["Topic Overlap"] = {
            "Common Topics": list(common_topics),
            "Unique to Article 1": list(unique_1),
            "Unique to Article 2": list(unique_2)
        }

        comparisons.append(comparison)

    return comparisons

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def calculate_final_sentiment(sentiment_distribution):
    """Determines the overall sentiment based on sentiment distribution."""
    max_sentiment = max(sentiment_distribution, key=sentiment_distribution.get)

    if sentiment_distribution[max_sentiment] == 0:
        return "The overall sentiment is neutral, with no strong positive or negative influence."
    elif max_sentiment == "Positive":
        return "The latest news coverage is mostly positive. Potential stock growth expected."
    elif max_sentiment == "Negative":
        return "The overall sentiment is negative. Caution is advised due to potential risks."
    else:
        return "The sentiment is balanced, with both positive and negative aspects equally present."


In [ ]:
def translate_final_sentiment_to_hindi(final_sentiment_analysis):
  """Translates the final sentiment analysis to Hindi."""
  translator = Translator()
  try:
    translated_text = translator.translate(final_sentiment_analysis, dest='hi').text
    return translated_text
  except Exception as e:
    print(f"Error during translation: {e}")
    return "Translation failed."

def text_to_speech(text, language='en'):
  """Converts text to speech using gTTS."""
    tts = gTTS(text=text, lang=language)
    tts.save("news_audio.mp3")
    return Audio("output.mp3", autoplay=False)

In [ ]:

def generate_output(company, max_articles=10):
    """Fetches news, analyzes sentiment, classifies topics, and structures the final JSON output with comparative coverage."""
    articles = fetch_news(company, max_articles)
    articles = analyze_sentiment(articles)
    articles = classify_topics(articles)
    sentiment_distribution = compute_sentiment_distribution(articles)
    coverage_differences = compare_coverage(articles)
    final_sentiment = calculate_final_sentiment(sentiment_distribution)
    translated_sentiment = translate_final_sentiment_to_hindi(final_sentiment)

    text_to_speech(translated_sentiment, language='hi')

    output = {
        "Company": company,
        "Articles": articles,
        "Comparative Sentiment Score": {
            "Sentiment Distribution": sentiment_distribution,
            "Coverage Differences": coverage_differences
        },
        "Final Sentiment Analysis": final_sentiment,
        "Audio": "[Play Hindi Speech]"
    }

    return(json.dumps(output, indent=4))
    display(Audio("news_audio.mp3", autoplay=True))


In [ ]:
# Take user input
if __name__ == "__main__":
    company_name = input("Enter the company name: ")
    print(generate_output(company_name))

Enter the company name: tesla
{
    "Company": "tesla",
    "Articles": [
        {
            "Title": "Tesla analyst says CEO Elon Musk is \u2018back in charge\u2019 after surprise all-hands meeting, and investors are rewarding the company\u2019s stock",
            "Summary": "Musk stepped up last week with the all-hands meeting, and that sent a much needed positive signal to employees and investors, ...",
            "Sentiment": "positive",
            "Topics": [
                "Leadership",
                "Technology",
                "Innovation"
            ]
        },
        {
            "Title": "Tesla Takedown: all the news about the protests roiling the automaker",
            "Summary": "Today we\u2019re talking about the protests against Tesla, which have been branded on social media under the hashtag Tesla ...",
            "Sentiment": "positive",
            "Topics": [
                "Technology",
                "Electric Vehicles",
                "Consumer 